In [0]:
%sql
CREATE OR REPLACE FUNCTION t3_a00cd1_snd_arg02_dbws01.ext_parquet.mask_for_all_roles(id DECIMAL)
RETURNS DECIMAL
RETURN  NULL
;

In [0]:
from databricks.sdk import WorkspaceClient
import json

w = WorkspaceClient()

securable_type = "CATALOG"
securable_full_name = "t3_a00cd1_snd_arg02_dbws01"
policy_name = "10MODERATED_CONTENT"

policy_payload={
 'on_securable_type': 'CATALOG',
 'on_securable_fullname': 't3_a00cd1_snd_arg02_dbws01',
 'name': '10MODERATED_CONTENT',
 'comment': 'Mask id column for all users except l205495 when MODERATED_CONTENT tag is present',
 'to_principals': ['account users'],
 'except_principals': ['l205495@sandpit0x8a.onmicrosoft.com'],
 'for_securable_type': 'TABLE',
 'rule_type': 'RULE_TYPE_COLUMN_MASK',
 'policy_type': 'POLICY_TYPE_COLUMN_MASK',
 'column_mask': {'function_name': 't3_a00cd1_snd_arg02_dbws01.ext_parquet.mask_for_all_roles',
  'on_column': 'm',
  'condition': "hasTag('MODERATED_CONTENT')"},
 'match_columns': [{'condition': "hasTag('MODERATED_CONTENT')", 'alias': 'm'}]}
 


# Delete policy if it exists
try:
    w.api_client.do(
        "DELETE",
        f"/api/2.1/unity-catalog/policies/{securable_type}/{securable_full_name}/{policy_name}"
    )
    print(f"Deleted existing policy: {policy_name}")
except Exception as e:
    if "does not exist" in str(e) or "RESOURCE_DOES_NOT_EXIST" in str(e) or "NotFound" in str(e):
        print(f"No existing policy to delete: {policy_name}")
    else:
        raise

# Create the policy fresh
print(f"Creating new policy: {policy_name}")
w.api_client.do(
    "POST",
    "/api/2.1/unity-catalog/policies/",
    data=json.dumps(policy_payload)
)
